<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/Module%204/logo.png" width="300" alt="cognitiveclass.ai logo" />
</center>

# Capstone Project week 3: Plotly Dashboard

<br>
I've decided to make this lab in Jupyter nootebook. Thank's to module JupyterDash it is possible and, in my opinion, easier to do. It aslo gives more possibilities like adding comments or just write a short summarization on found insights - which I will include in the bootom of the file.
<br>
<br>

In [6]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [13]:
spacex_df.columns

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'Mission Outcome',
       'class', 'Payload Mass (kg)', 'Booster Version',
       'Booster Version Category'],
      dtype='object')

In [40]:
grp = spacex_df[['Launch Site', 'class']]

In [46]:
grp1 = grp.groupby('Launch Site').sum()

In [47]:
grp1

,class
Launch Site,
CCAFS LC-40,7
CCAFS SLC-40,3
KSC LC-39A,10
VAFB SLC-4E,4


In [12]:
spacex_df['Launch Site'].value_counts()

CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: Launch Site, dtype: int64

In [19]:
# Create a dash application
app = dash.Dash(__name__)

In [28]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                ],
                                                value='ALL',
                                                placeholder="Select a Launch Site here",
                                                searchable=True
                                                ),                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       100: '100'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [21]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

In [22]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='title')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', names='class',
        title=f'Total Success Launches for Site {entered_site}')
        return fig

In [ ]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

In [31]:
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_plot(entered_site, slider_range):
    low, high = slider_range
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    filtered_df1 = spacex_df[mask]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df1, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Correlation Between Payload and Success for All Sites')
        return fig
    else:
        filtered_df2 = filtered_df1[filtered_df1['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df2, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title=f'Correlation Between Payload and Success for Site {entered_site}')
        return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

# Insights 

1. Which site has the largest successful launches?

    KSC LC-39a
    
    
2. Which site has the highest launch success rate?

    CCAFS SLC-40  -  42.9% of success launch
    
    
3. Which payload range(s) has the highest launch success rate?

    Around 2000kg to 5300kg
    
    
4. Which payload range(s) has the lowest launch success rate?

    Definitely about 6000kg. But it's hard to say as in ranges 2k-4k and 4k-6k there are similar number of failed launches. 
    

5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest lauch success rate?

    Booster FT.
    